In [2]:
import numpy as np
import glob
import xarray as xr 
import matplotlib.pyplot as plt
from collections import namedtuple
import pandas
import glob
import sys, os
sys.path.insert(0, os.path.abspath('./lib'))

from lib import read_xarray, find_neighbours,create_new_mask, conversion
from lib import hss,precision,far,f1

In [3]:
def get_WME_legend(file_CodesWWMF, ds):
    '''return: legend and WME code which are predicted in ds
    '''
    df             = pandas.read_csv(file_CodesWWMF,usecols = (0,1,6,7),sep=',')
    Code_WME       = df['Code WME'].to_numpy()
    Code_WWMF      = df['Code WWMF'].to_numpy()
    legende_WWMF   = df['Legende WWMF'].to_numpy()
    legende_WME    = df['Legende WME'].to_numpy()

    cible_list  = np.unique(ds.wme_arr.values[~np.isnan(ds.wme_arr.values)])# on parcourt les temps sensibles disponibles sur la zone
    legend_list = [np.unique(legende_WME[Code_WME == WME])[0] for WME in cible_list] 
    return cible_list,legend_list

In [4]:
# ecriture d'une fonction qui cree bcp de combinaisons possible de masks

def create_combination_subzones(dir_mask,dep_id,lst_subzones,fname_out,degre5=False):
    '''
    calcul et creation d'une multitude de combinaison de zones sympos. 
    in:
        - dir_mask: répertoire de là où sont stocké les fichiers des zones sympos
        - dep_id: numéro du département (38 dans le cas de l'Isère)
        - lst_subzones: liste de subzones de zones sympos dans le département 
        - fname_out: fichier où sont stockés tous les différents masks
        - degre5: si on veut des masks à 5 zones aussi 
    '''
    
    ds_mask = xr.Dataset()
    '''
       1 - Mask global: contient la somme des zones sympos
    '''
    for id in lst_subzones:
    #for n in range(1,n_subzones+1):
        file_2 = dir_mask+id+'.nc' #.format(dep_id,n)   
        ds_2   = read_xarray(file_2)
        ds_2 = ds_2.reset_index("id",drop=True)# pour pouvoir les ajouter 
        ds_mask = xr.merge([ds_mask,ds_2],join='outer')
    ds_mask = ds_mask.assign_coords(id = ['departement'])
    ds_mask.mask.attrs["name"] = 'combinaisons des zones sympos'

    '''
       2 - Mask de chaque zone sur le grand domaine 'departement' 
    '''
#     for n in range(1,n_subzones+1):
    for id in lst_subzones:
        file_2 = dir_mask+ id + '.nc' #'{}{:02d}.nc'.format(dep_id,n)   
        ds_2   = read_xarray(file_2)
        new_id = ds_2.id.values[0]
        if new_id in ds_mask.id.values: 
            continue
        ds_3 = ds_2 *ds_mask.sel(id="departement") 
        ds_3 = ds_3.assign_coords(id = [new_id]) 
#         ds_3 = ds_3.where((ds_3.mask.values==1) & (np.isnan(ds_mask.sel(id='departement').mask.values)),0)
        ds_mask  = xr.concat([ds_mask,ds_3],dim = 'id')
        del(ds_3,ds_2)
        
    '''
       3 - Mask des somme des zones sympos voisines: return des mask du type 'zone1+zone3'
    '''
    lst_id_before3 = ds_mask.id.values
    for id_ref in lst_subzones:
        mask_ref = ds_mask.sel(id = id_ref).copy(deep=True)
        # on recupère toutes les zones voisines à la zone ref
        listMasks = [ds_mask.sel(id = id) for id in lst_subzones]

        list_neighbours = find_neighbours(mask_ref,listMasks)
    #     print("{} voisin avec {}".format(mask_ref.id.values,list_neighbours))
    #     on crée les nouveaux mask qui englobent mask_ref + voisin
        listMasks = [ds_mask.sel(id = id) for id in list_neighbours]  
        mask_ref = ds_mask.sel(id = id_ref).copy(deep=True)
        ds_mask = create_new_mask(ds_mask, id_ref,listMasks)

    '''
       4 - somme entre chaque groupement de zones sympos: 
       return des masks du type '(zone1 + zone3) + (zone4 + zone5)' ou '(zone1 + zone3) + (zone4 )'
    '''
    lst_int4 = np.copy(ds_mask.id.values)
    lst_new_id4 = [key for key in lst_int4 if key not in lst_id_before3]

#     print(lst_new_id)
    for id_ref in lst_int4: #lst_new_id: 
        mask_ref = ds_mask.sel(id = id_ref).copy(deep=True) 
        listMasks = [ds_mask.sel(id = id) for id in lst_int4]
        list_neighbours = find_neighbours(mask_ref,listMasks)
#         print("{} voisin avec {}".format(id_ref,list_neighbours))
        if len(list_neighbours) == 0: 
            continue
        listMasks = [ds_mask.sel(id = id) for id in list_neighbours]  
        mask_ref  = ds_mask.sel(id = id_ref).copy(deep=True)
        ds_mask   = create_new_mask(ds_mask, id_ref,listMasks)
  
    '''
        à voir si on veut continuer sur cette boucle ou pas
    '''
    if degre5:
        lst_int5 = np.copy(ds_mask.id.values)
        lst_new_id5 = [key for key in lst_int5 if key not in lst_int4]

        for id_ref in lst_new_id5: #lst_new_id: 
            mask_ref = ds_mask.sel(id = id_ref).copy(deep=True) 
            listMasks = [ds_mask.sel(id = id) for id in lst_int5]
            list_neighbours = find_neighbours(mask_ref,listMasks)
    #         print("{} voisin avec {}".format(id_ref,list_neighbours))
            if len(list_neighbours) == 0: 
                continue
            listMasks = [ds_mask.sel(id = id) for id in list_neighbours]  
            mask_ref  = ds_mask.sel(id = id_ref).copy(deep=True)
            ds_mask   = create_new_mask(ds_mask, id_ref,listMasks)
#         print()
#     pour avoir des 0 en dehors de la sous-zone sur le departement 
    for iid,id_ref in enumerate(ds_mask.id.values): 
        ds_mask.mask[iid,:,:] = ds_mask.sel(id=id_ref).mask.where((ds_mask.sel(id=id_ref).mask.values==1) + (np.isnan(ds_mask.sel(id='departement').mask.values)),0)

    ds_mask.to_netcdf(fname_out)
    return ds_mask



In [5]:
def create_nc_mask_NSEO(dep_file,fname_mask_NSEO,plot_dep=False):
    '''
        fonction qui divise le mask (dep_file) en une vingtaine de sous-zones géographiques (nord,sud, nord  + est, etc...)
        et sauvegarde ce nouveau mask en ncdf. 
        in : dep_file
        out: le mask avec les sous-zones 
        
        !!! fonction à rendre plus claire !!! 
    '''
    ds_dep    = read_xarray(dep_file) 

    lat1_3,lat2_3 = ds_dep["latitude"].quantile([1/3,2/3]) #,dim=["latitude"])
    lon1_3,lon2_3 = ds_dep["longitude"].quantile([1/3,2/3]) #,dim=["latitude"])

    ds_mask = ds_dep.copy().squeeze("id") # on va creer un dataset qui contient les masks N, S, E, O
    latmin_dict = {'nord':float(lat2_3.values),'sud':ds_mask.latitude.values.min()
                   ,'ouest':ds_mask.latitude.values.min(),'est':ds_mask.latitude.values.min()}

    latmax_dict = {'nord':ds_mask.latitude.values.max(),'sud':float(lat1_3.values)
                   ,'ouest':ds_mask.latitude.values.max(),'est':ds_mask.latitude.values.max()}

    lonmin_dict = {'nord':ds_mask.longitude.values.min(),'sud':ds_mask.longitude.values.min()
                   ,'ouest':ds_mask.longitude.values.min(),'est':float(lon2_3.values)}

    lonmax_dict = {'nord':ds_mask.longitude.values.max(),'sud':ds_mask.longitude.values.max()
                   ,'ouest':float(lon1_3.values),'est':ds_mask.longitude.values.max()}

    # definition des masks centraux
    for imask_id,mask_id in enumerate(['nord','sud','est','ouest']):
        ds_mask[mask_id] = ds_mask.mask * 0
        ind    = np.where((ds_mask.latitude>=latmin_dict[mask_id]) & (ds_mask.latitude<=latmax_dict[mask_id])
                 & (ds_mask.longitude >= lonmin_dict[mask_id]) & (ds_mask.longitude<=lonmax_dict[mask_id]))
        ds_mask[mask_id].values[ind] = 1
        ds_mask[mask_id].values[np.isnan(ds_dep.squeeze("id").mask.values)] = np.nan

    # definition des masks croises
    for var1 in ['est','ouest']:
        for var2 in ['nord','sud']: 
            mask_id = var2 + '-'+var1
            ds_mask[mask_id] = ds_mask[var1]* ds_mask[var2] # egal a 1 quand les deux valent 1 

    # def du central 
    mask_id = 'centre'
    ds_mask[mask_id] = ds_mask.mask.copy()
    ds_mask[mask_id].values[(ds_mask.nord ==1 ) + (ds_mask.sud ==1 ) 
                            + (ds_mask.est ==1 ) + (ds_mask.ouest ==1 )] = 0
    # Nord + Est et Nord + ouest (idem pour sud)
    for var1 in ['nord','sud']:
        for var2 in ['est','ouest']: 
            mask_id = var1 + '+'+var2
            ds_mask[mask_id] = ds_mask[var1].copy()
            ds_mask[mask_id].values[ds_mask[var2].values==1] = 1
    # centre + nord (resp + sud, + est, + ouest)
    for var in ['nord','sud','est','ouest']:
        mask_id = 'centre+'+var
        ds_mask[mask_id] = ds_mask['centre'].copy()
        ds_mask[mask_id].values[ds_mask[var].values==1] = 1

    # tout le mask sauf certaines zones 
    for var in ['nord','sud','est','ouest']:
        mask_id = 'tout-'+var
        ds_mask[mask_id] = ds_mask.mask.copy() - ds_mask[var].values
    #     ds_mask[mask_id].values[ds_mask[var].values==1] = 1

    if plot_dep:
        fig,axes = plt.subplots(nrows=5,ncols =4,figsize=(15,10))
        ax = axes.flat
        for imask_id,mask_id in enumerate(['nord','sud','est','ouest'                                      
                                           ,'nord-est','sud-est','nord-ouest','sud-ouest'
                                            ,'nord+est','sud+est','nord+ouest','sud+ouest'
                                           ,'centre+est','centre+nord','centre+ouest','centre+sud'
                                           ,'tout-est','tout-nord','tout-ouest','tout-sud'
                                          ]): #['nord','sud','est','ouest']):
            ds_mask[mask_id].plot.imshow(ax=ax[imask_id])
            ax[imask_id].set_title(mask_id)
        fig.tight_layout()

    # save 
    ds_out = xr.Dataset()
    for i,keys in enumerate(ds_mask.data_vars):
        ds_temp = ds_mask[keys].expand_dims("id").assign_coords(id=[keys]).rename("mask")
        ds_out = xr.merge([ds_out,ds_temp])
    ds_out.to_netcdf(fname_mask_NSEO)
    return ds_out 

# création ou lecture des zones sympos sur le departement en questions 

In [6]:
# 29 : Finistère (185), 34: Hérault (235 combi), 38: Isère (80 combi), 41: Loi-et-cher (27)
import time
dep_id = '34' #'29'#'41' #'38'#
mask_sympo = True
mask_geographique = False

# mask_sympo = False
# mask_geographique = True


if mask_sympo and not mask_geographique: 
    dir_mask = '/home/mrpa/borderiesm/stageEMI/Codes/StageEMI/Masques_netcdf/ZONE_SYMPO/'
    list_subzones = glob.glob(dir_mask + dep_id +'*.nc')
    n_subzones = len(list_subzones)  # nombre de zones sympos initiales
    lst_subzones = [zone[-7:-3] for zone in list_subzones]
    t1 = time.time()
    fname_out = '/home/mrpa/borderiesm/stageEMI/Codes/StageEMI/Masques_netcdf/ZONE_SYMPO_MULTIPLE/'+ dep_id+'_mask_zones_sympos.nc'
#     ds_mask = create_combination_subzones(dir_mask,dep_id,lst_subzones,fname_out) 
    if not os.path.exists(fname_out): 
        ds_mask = create_combination_subzones(dir_mask,dep_id,lst_subzones,fname_out,degre5=True) 
    else: 
        ds_mask = read_xarray(fname_out)
    print(time.time() - t1)
    
if mask_geographique and not mask_sympo: 
    dir_mask  = '/home/mrpa/borderiesm/stageEMI/Codes/stageemi/stageemi/GeoData/nc_departement/'
    if   dep_id == '38': dep = 'FRK24'
    elif dep_id == '41': dep = 'FRB05'
    elif dep_id == "34": dep = 'FRJ13'
    elif dep_id == '29': dep = "FRH02"
    else: 
        print('remplir la bonne valeur pour le dep')
        sys.exit()
        
    dep_file  = dir_mask + dep +'.nc' 
    fname_out = '/home/mrpa/borderiesm/stageEMI/Codes/StageEMI/Masques_netcdf/ZONE_SYMPO_MULTIPLE/'+ dep_id+'_'+dep+'_mask_NSEO.nc'
    if not os.path.exists(fname_out):
        ds_mask = create_nc_mask_NSEO(dep_file,fname_out)
    else:
        ds_mask = read_xarray(fname_out) 
    
print(ds_mask.id.values,ds_mask.id.values.size)

21829.152678489685
['departement' '3401' '3402' ... '3403+3404+3406+3407+3408+3483+3481+3482'
 '3403+3406+3405+3407+3408+3483+3481+3482'
 '3404+3405+3406+3407+3408+3483+3481+3482'] 1257


# Lecture du champs Arome et visualisation du champs sur le departement

In [ ]:
dir_in = '/scratch/labia/lepapeb/StageEMI/WWMF/'
date   = '2019110400'
fname  = dir_in + date+'0000__PG0PAROME__'+'WWMF'+'__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc'
ds     = read_xarray(fname)

if mask_sympo and not mask_geographique: 
    ds2plot = ds.isel(step=17) * ds_mask.mask.sel(id='departement') 
if mask_geographique and not mask_sympo: 
    ds2plot = ds.isel(step=17) * ds_mask.mask.sel(id="mask") 
# ds2plot.unknown[ind] = 0 
ds_WME = conversion(ds2plot) 
ds_WME.wme_arr.values[ds_WME.wme_arr.values<4] = 2 
ds_WME.wme_arr.values[ds_WME.wme_arr.values>=4] = 3 

del(ds,ds2plot)
ds_WME.wme_arr.plot.imshow()


In [ ]:
file_CodesWWMF = '../../utils/CodesWWMF.csv'
cible_list,legend_list = get_WME_legend(file_CodesWWMF, ds_WME)
print(cible_list,legend_list)

# groupement des masks par taille (à voir si c'est vraiment nécessaire ??)

In [ ]:
taille_masks = ds_mask.mask.sum(["longitude","latitude"]) # /ds_mask.mask.sel(id=['mask']).sum()
taille1,taille2 = taille_masks.quantile([1/3,2/3])

print(taille1.values,taille2.values)
ind1 = np.where( (taille_masks.values < taille1.values) & (taille_masks.values > 0))
groupe1 = ds_mask.mask.isel(id=ind1[0])

ind2 = np.where((taille_masks.values >= taille1.values) & (taille_masks.values < taille2.values))
groupe2 = ds_mask.mask.isel(id=ind2[0])

ind3 = np.where((taille_masks.values >= taille2.values))
groupe3 = ds_mask.mask.isel(id=ind3[0])

print(groupe1.id.values,'\n')
print(groupe2.id.values,'\n')
print(groupe3.id.values)
print(ds_mask.id.values.size)



# Recherche de la sous-zone optimale

In [ ]:
cible =  3
# recherche de la sous-zone optimale
taille_objet_binaire = np.sum(ds_WME.wme_arr.values==cible)
print('taille objet binaire ', taille_objet_binaire)
if taille_objet_binaire < taille1.values and taille_objet_binaire >0: # groupe 1 de masks
    print('objet dans groupe 1')
    groupe_mask_select = groupe1

elif taille_objet_binaire>=taille1.values and taille_objet_binaire < taille2.values: # groupe 2 de mask
    print('objet dans groupe 2')
    groupe_mask_select = groupe2
else :
    # On parcourt la liste de mask, calcul dun critere d'homogeneite dans le groupe 
    print('objet dans groupe 3')
    groupe_mask_select = groupe3

# groupe_mask_select =  ds_mask.mask# groupe2    
# groupe_mask_select = ds_mask.mask.sel(id=['3801','3807']) #(id=['sud-est','sud']) #(id=['3801','3807'])
# groupe_mask_select = ds_mask.mask.sel(id=['sud-est','sud']) 

score_precision = np.zeros(len(groupe_mask_select))    
score_f1        = np.zeros(len(groupe_mask_select)) 
score_hss       = np.zeros(len(groupe_mask_select)) 
score_far       = np.zeros(len(groupe_mask_select))

for imask,ds_mask_sub in enumerate(groupe_mask_select):    
    # check are aranged in the same order 
    lat1 = ds_mask_sub.latitude.values
    lat2 = ds_WME.latitude.values
    if (np.sum(lat1==lat2) == lat1.size ): 
        # same order 
#         print("same order")
        y_true = ds_WME.wme_arr.copy()
    elif (np.sum(lat1[::-1]==lat2)== lat1.size):
#         print('reverse order ')
        y_true = ds_WME.wme_arr[::-1,:].copy()
    else: 
        print("pb sur lon/lat")
        break
        
    y_pred = ds_mask_sub.copy() 
    y_true.values[(y_true.values!=cible) & (~np.isnan(y_true.values))] = 0 
    y_true.values[y_true.values == cible ] = 1 
    
   
    y_true_score = y_true.values[~np.isnan(y_true.values)]
    y_pred_score = y_pred.values[~np.isnan(y_pred.values)]
    # metriques : 
    score_precision[imask] = precision(y_true_score,y_pred_score)
    score_f1[imask]        = f1(y_true_score,y_pred_score)
    score_hss[imask]       = hss(y_true_score,y_pred_score)
    score_far[imask]       = far(y_true_score,y_pred_score)
    
print('pour la cible {}'.format(cible))
ind_nan = np.where(~np.isnan(score_precision))
indice_groupe_select = np.argsort(score_precision[ind_nan])[::-1][:2] 
print('zone majoritaire occupee pour cible {}: {} ({})'.format(cible,                
            groupe_mask_select.id.values[ind_nan][indice_groupe_select],
            score_precision[ind_nan][indice_groupe_select]))

ind_nan = np.where(~np.isnan(score_f1))
indice_groupe_select_f1 = np.argsort(score_f1[ind_nan])[::-1][:2] #np.nanargmax(score_f1)
print('zone F1 occupee pour cible {}: {} ({})'.format(cible,
                groupe_mask_select.id.values[ind_nan][indice_groupe_select_f1],
                score_f1[ind_nan][indice_groupe_select_f1]))    

ind_nan = np.where((~np.isnan(score_far)) & (score_far>0))
indice_groupe_select_far = np.argsort(score_far[ind_nan])[:2] #np.nanargmin(score_far)
print('zone far occupee pour cible {}: {} ({})'.format(cible,
                groupe_mask_select.id.values[ind_nan][indice_groupe_select_far], 
                score_far[ind_nan][indice_groupe_select_far])) 

ind_nan = np.where(~np.isnan(score_hss))
indice_groupe_select_hss = np.argsort(score_hss[ind_nan])[::-1][:2]#np.nanargmax(score_hss)
print('zone hss occupee pour cible {}: {} ({})'.format(cible,
                groupe_mask_select.id.values[ind_nan][indice_groupe_select_hss],
                score_hss[ind_nan][indice_groupe_select_hss])) 
print()    

# score_far[np.argsort(score_far)[::-1]]
# ind = np.where(np.isnan(score_f1))

In [ ]:
cible = 3
# groupe_mask_select = ds_mask.mask.sel(id=['3801','3807']) #(id=['sud-est','sud']) #(id=['3801','3807'])
groupe_mask_select = ds_mask.mask.sel(id=['sud-est','sud']) 
# sys.exit()    
score_precision = np.zeros(len(groupe_mask_select))    
score_f1        = np.zeros(len(groupe_mask_select)) 
score_hss       = np.zeros(len(groupe_mask_select)) 
score_far       = np.zeros(len(groupe_mask_select))

for imask,ds_mask_sub in enumerate(groupe_mask_select):    
    
    
    
    print(y_true.shape)
    y_true = ds_WME.wme_arr[::-1,:].copy()
#     y_true = y_true[::-1,:]
    y_pred = ds_mask_sub.copy() 
    y_true.values[(y_true.values!=cible) & (~np.isnan(y_true.values))] = 0 
    y_true.values[y_true.values == cible ] = 1 
#     ind = np.where(~np.isnan(y_pred.values))
#     y_true.values[ind] = 8
#     if imask == 0:
#         fig,axes= plt.subplots(nrows=1,ncols=2)
#         y_true.plot(ax=axes[0])        
#         y_pred.plot(ax=axes[1])        
#         axes[0].set_title("")
#         axes[1].set_title("")
# #         tp = np.sum((y_true==3)*(y_pred==1))
#         ind = np.where((y_true.values==1) * (y_pred.values==1))
# #         ind = np.where((y_pred.values==1))

#         print(y_pred.values[ind])

lat1 = ds_mask_sub.latitude.values
lat2 = ds_WME.latitude.values
# print(lat1,lat2)
print(np.sum(lat1[::-1]==lat2))
print(np.sum(lat1==lat2) == lat1.size)


In [ ]:
print(ds_mask.sel(id='3801').mask.values.shape)
print(ds_WME.wme_arr.values.shape)

In [ ]:
cible = 3
# on a pas de tp sur la zone 
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(15,5))
ds_int = ds_WME.wme_arr * ds_mask.mask.sel(id='departement')

ds_int.plot(ax=ax[0])
ds_3 = ds_int.copy(deep=True)
print(ds_3.values.shape)
ds_3.values[(ds_3.values!=cible) & (~np.isnan(ds_3.values)) ] = 0
ds_3.values[ds_3.values==cible] = 1 

# ind = np.where(ds_mask.sel(id='3801').mask.values==1)
# ds_3.values[ind] = 5
# y.values[np.isnan(ds_WME.wme_arr.copy())] = np.nan

ds_3.plot(ax=ax[1])

# ds_mask.sel(id='3801').mask.plot(ax=ax[2])
ds_4 = ds_mask.sel(id='3801').mask.copy()
ds_4.values[np.isnan(ds_mask.sel(id='3801').mask.values)]=np.nan
ds_4.plot(ax=ax[2])
print(ds_4.values.shape)

y_pred = ds_4.values[~np.isnan(ds_4.values)]
y_true = ds_3.values[~np.isnan(ds_3.values)]

# (ds_WME.wme_arr * ds_mask.sel(id='3801').mask).plot()
# y_pred  = #ds_mask.sel(id='3801').mask.values.flatten().copy()

# ind     = np.where((~np.isnan(y_pred)) ) # & (~np.isnan(y.values.flatten())) )    
# if ind[0].size==0 : 
#     print('pb')
# y_pred  = y_pred[ind]
# y_true  = y.values.flatten()[ind]

# print(y_true)
# # y_true[y_true !=cible] = 0
# # y_true[y_true ==cible] = 1

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true,y_pred).ravel()
print(tn, fp, fn, tp)

# somme = np.sum((y_true == 1) & (y_pred == 1))
# print(somme)
# print(f1(y_true,y_pred))

# # metriques : 
# score_precision[imask] = precision(y_true,y_pred)
# score_f1[imask]        = f1(y_true,y_pred)

In [ ]:
print(ds_mask.id.values.size)
ds_out = ds_mask.copy()
fig,axes = plt.subplots(nrows = 10,ncols=8,figsize = (20,15))
ax = axes.flat
for i,id in enumerate(ds_out.id.values):
    ds_out.sel(id=id).mask.plot(ax=ax[i])
    ax[i].set_title(id)
fig.tight_layout()